In [8]:
def gen_chapters(settings, section_id):
    section = next((section for section in settings['sections'] if section['id'] == section_id), None)
    response = openai.chat.completions.create(
        model= settings["model"],
        messages= [
            {
                "role": "system",
                "content": 
                    f"""
                    You are an expert historian that specializes in building book projects.
                    Your task is to generate the chapters of the provided section of a book.
                    The tone of voice for the book is: {settings['tone_of_voice']}.
                    The length for the excerpts is between {settings['excerpt_length_min']} and {settings['excerpt_length_max']} words.
                    """
            },
            {
                "role": "system",
                "content": 
                    """
                    The output is a JSON document like:

                    {
                      "chapters": [
                        {
                          "id": 1,
                          "title": "Title of chapter 1...",
                          "excerpt": "Excerpt of chapter 1..."
                        },
                        {
                          "id": 2,
                          "title": "Title of chapter 2...",
                          "excerpt": "Excerpt of chapter 2..."
                        }
                      ]
                    }

                    The ID of the chapter is a progressive number that is global to the entire book.
                    Use the last available id from the full outline to calculate the next chapter id.
                    """
            },
            {
                "role": "user",
                "content":
                    f"""
                    This is the general information about the book i'm writing:

                    Subject: {settings['subject']}
                    Title: {settings['title']}
                    Subtitle: {settings['subtitle']}

                    This is the full outline of the book so far:
                    (use this content to generate the next chapter's ID)
                    {settings['sections']}

                    The section of the book that you should develop is:
                    {section}

                    Generate between {settings['chapters_min']} and {settings['chapters_max']} chapters for the provided section.
                    """
            }
        ]
    )
    data = text2json(response.choices[0].message.content)
    return data